Text-to-speech (TTS) is the task of creating natural-sounding speech from text, where the speech can be generated in multiple languages and for multiple speakers. Several text-to-speech models are currently available in 🤗 Transformers, such as Bark, MMS, VITS and SpeechT5.

You can easily generate audio using the "text-to-audio" pipeline (or its alias - "text-to-speech"). Some models, like Bark, can also be conditioned to generate non-verbal communications such as laughing, sighing and crying, or even add music.

Here’s an example of how you would use the "text-to-speech" pipeline with Bark:

In [ ]:
from transformers import pipeline
from IPython.display import Audio

pipe = pipeline("text-to-speech", model="suno/bark-small")
text = "[clears throat] This is a test ... and I just took a long pause."
output = pipe(text)
Audio(output["audio"], rate=output["sampling_rate"])


If you are looking to fine-tune a TTS model, the only text-to-speech models currently available in 🤗 Transformers are SpeechT5 and FastSpeech2Conformer, though more will be added in the future. SpeechT5 is pre-trained on a combination of speech-to-text and text-to-speech data, allowing it to learn a unified space of hidden representations shared by both text and speech. This means that the same pre-trained model can be fine-tuned for different tasks. Furthermore, SpeechT5 supports multiple speakers through x-vector speaker embeddings.

The remainder of this guide illustrates how to:
1. Fine-tune SpeechT5 that was originally trained on English speech on the Dutch (nl) language subset of the VoxPopuli dataset.
2. Use your refined model for inference in one of two ways: using a pipeline or directly.

# Libraries

In [ ]:
pip install datasets soundfile speechbrain accelerate

# Install 🤗Transformers from source as not all the SpeechT5 features have been merged into an official release
pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# To follow this guide you will need a GPU

# If you’re working in a notebook, run the following line to check if NVIDIA GPU available
!nvidia-smi

# Or for AMD GPU
!rocm-smi

# Data

VoxPopuli is a large-scale multilingual speech corpus consisting of data sourced from 2009-2020 European Parliament event recordings. It contains labelled audio-transcription data for 15 European languages. In this guide, we are using the Dutch language subset, feel free to pick another subset.

Note that VoxPopuli or any other automated speech recognition (ASR) dataset may not be the most suitable option for training TTS models. The features that make it beneficial for ASR, such as excessive background noise, are typically undesirable in TTS. However, finding top-quality, multilingual, and multi-speaker TTS datasets can be quite challenging.

In [ ]:
# Load data set
from datasets import load_dataset, Audio

# Check len == 20968 examples
dataset = load_dataset("facebook/voxpopuli", "nl", split="train")
len(dataset)

In [ ]:
# SpeechT5 expects audio data to have a sampling rate of 16 kHz
# Make sure the examples in the dataset meet the requirement of 16kHz sampling rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))